In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.multicomp as mc
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
import statsmodels.api as sm
import scipy.stats as stats
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.metrics import mean_squared_error
import re
from joblib import dump,load

In [ ]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

In [ ]:
df_main=pd.read_csv(r'village_wise_survey_data_mission_antyodaya_2020.csv')

In [ ]:
df_main.drop(columns=["SUB DISTRICT CODE","BLOCK CODE","GP CODE","VILLAGE CODE","VILLAGE PIN CODE","STATE NAME", "DISTRICT NAME","SUB DISTRICT NAME","BLOCK NAME","GP NAME","VILLAGE NAME","PC CODE","AC CODE","OTHER ASSEMBLY CONSTITUENCIES","NUMBER OF HOUSEHOLDS ENGAGED MAJORLY IN NON-FARM ACTIVITIES"],
             inplace=True)
df_main.head()

In [ ]:
cols=df_main.columns.tolist()
for i in range(len(cols)):
    if cols[i]=="DOES THE VILLAGE HAVE LIVESTOCK EXTENSION SERVICES":
        print(i)
        endcol=i

In [ ]:
df_main.columns.to_list()

In [ ]:
df=df_main.iloc[:,:endcol]

In [ ]:
df.head()

In [ ]:
cols=df.columns.tolist()


df["Percentage_Of_Farming_Houses"]=df["NUMBER OF HOUSEHOLDS ENGAGED MAJORLY IN FARM ACTIVITIES"]/(df["NUMBER OF TOTAL HOUSEHOLD"])
df["DOES THE VILLAGE HAS ANY FARMERS COLLECTIVE"]=df["DOES THE VILLAGE HAS ANY FARMERS COLLECTIVE"].map({"Primary Agriculture Cooperative Society(PACS)":"PACS","Farmers Produce Organization(FPOs)":"FPO","Both":"Both"})
df["MAIN SOURCE OF IRRIGATION"]=df["MAIN SOURCE OF IRRIGATION"].map({"Ground water (tube well/well/pump)":"Ground Water","Other":"Other","Canals":"Canals","Surface Water":"Surface Water"})


In [ ]:
for i in tqdm(range(len(df))):
    string = str(df["NET SOWN AREA (IN HECTARES) , IF IN ACRES DIVIDE BY 2.47"][i])
    numbers = re.findall(r'\d+\.*\d*', string)

    df.loc[i,'Area'] =numbers[0]
    df.loc[i,'Kharif'] =numbers[1]
    df.loc[i,'Rabi'] =numbers[2]
    df.loc[i,'Others'] =numbers[3]

In [ ]:
cols = ['availability of warehouse for food grain storage ','availability of soil testing centres','availability of fertilizer shop']
dic = dict(zip(df['availability of fertilizer shop'.upper()].value_counts().index.to_list(),['>10','5-10','2-5','Yes','1-2','<1']))


for col in cols:
    col=col.upper()
    df[col] = df[col].map(dic)


df=df.drop(columns="NET SOWN AREA (IN HECTARES) , IF IN ACRES DIVIDE BY 2.47")

In [ ]:
cols = df.columns.to_list()
new_cols = [re.sub(r'[^a-zA-Z0-9_]' , '_', a.strip().lower()) for a in cols]
df.rename(columns=dict(zip(cols, new_cols)), inplace=True)
df.rename(columns=dict(zip(cols, new_cols)), inplace=True)
df.rename(columns={'total_area_irrigated__in_hectare___if_in_acres_divide_by_2_47': 'total_area_irrigated__in_hectare'},inplace=True)
df.rename(columns={'total_unirrigated_land_area__in_hectares___if_in_acres_divide_by_2_47': 'total_unirrigated_land_area__in_hectares'},inplace=True)

In [ ]:
for col in df.columns:
    if df[col].isna().sum()!=0:
        print(col)

In [ ]:
df.fillna('None',inplace=True)

In [ ]:
for col in df.columns:
    if df[col].isna().sum()!=0:
        print(col)

In [ ]:
df.to_csv("Agriculture.csv", index=False)

In [ ]:
df=pd.read_csv("Agriculture.csv")
df.head()

In [ ]:
string = "+".join(df.drop('number_of_households_engaged_majorly_in_farm_activities',axis=1).columns.to_list())

In [ ]:
string

In [ ]:
fit1 = ols(f'number_of_households_engaged_majorly_in_farm_activities~{string}',data=df).fit()
p_value = sm.stats.anova_lm(fit1,typ=1)

In [ ]:
p_value_sig=p_value[p_value['PR(>F)']<0.05].sort_values(by='PR(>F)', ascending=True)
cols_sig=p_value_sig.index.tolist()

In [ ]:
cols_sig.append("number_of_households_engaged_majorly_in_farm_activities")
cols_sig

In [ ]:
df_sig = df[cols_sig]
df_sig.head()

In [ ]:
df_sig.describe().T

In [ ]:
df_sig.info()

In [ ]:
for col in df_sig.columns:
    if pd.api.types.is_object_dtype(df_sig[col]):

        print(df_sig[col].value_counts())
        print()

In [ ]:
df1 = df_sig.copy()

In [ ]:
dummy_cols = [col for col in df1.columns if pd.api.types.is_object_dtype(df1[col])]
dummy_df = pd.get_dummies(df1, columns=dummy_cols)
df1 = pd.concat([df1,dummy_df], axis=1)
df1.drop(dummy_cols,inplace=True, axis= 1)
df1.info()

In [ ]:
X = df1.drop('number_of_households_engaged_majorly_in_farm_activities', axis=1)
y = df1['number_of_households_engaged_majorly_in_farm_activities']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
LinearRegression = LinearRegression()
Ridge = Ridge()
Lasso = Lasso()
ElasticNet = ElasticNet()
RandomForestRegressor = RandomForestRegressor(n_jobs=-1)

In [ ]:
algos = [LinearRegression ,Ridge ,Lasso, ElasticNet, RandomForestRegressor]

In [ ]:
scores = []
mses = []
algo_names = []

for algo in tqdm(algos):
    algo.fit(X_train,y_train)
    algo_name = get_var_name(algo)
    dump(value=algo,filename=f'{algo_name}.joblib')
    y_pred = algo.predict(X_test)
    acc_score = algo.score(X_test,y_test)
    mse = mean_squared_error(y_test,y_pred)
    algo_names.append(algo_name)
    scores.append(acc_score)
    mses.append(mse)

algo_scores = pd.DataFrame(
    {
        'Algorithm' : algo_names,
        'Score' : scores,
        'MSE' : mses
    }
)

In [ ]:
scores = []
mses = []
algo_names = []

for alg in tqdm(algos):
    algo_name = get_var_name(alg)
    algo = load(filename=f'{algo_name}.joblib')
    y_pred = algo.predict(X_test)
    acc_score = algo.score(X_test,y_test)
    mse = mean_squared_error(y_test,y_pred)
    algo_names.append(algo_name)
    scores.append(acc_score)
    mses.append(mse)

algo_scores = pd.DataFrame(
    {
        'Algorithm' : algo_names,
        'Score' : scores,
        'MSE' : mses
    }
)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(15, 10))
sns.barplot(
    x='Algorithm',
    y='Score',
    data=algo_scores,
    ax=axs[0],
    palette='viridis',
    legend=False,
    hue='Algorithm'
)
for i in axs[0].containers:
    axs[0].bar_label(i,)
sns.barplot(
    x='Algorithm',
    y='MSE',
    data=algo_scores,
    ax=axs[1],
    palette='mako',
    legend=False,
    hue='Algorithm'
)
for i in axs[1].containers:
    axs[1].bar_label(i,)
plt.show()

In [ ]:
for alg in tqdm(algos):
    alg = load(filename=f'{get_var_name(alg)}.joblib')